In [1]:
# Read sylph results
import pandas as pd

sylph_results = pd.read_csv("/mnt/c/Users/zhenh/tax_data/profile.tsv", sep='\t')

In [2]:
# Read GTDB metadata
import pandas as pd

gtdb_metadata = pd.read_csv('/mnt/c/Users/zhenh/tax_data/bac120_metadata.tsv',sep='\t')

/tmp/ipykernel_235218/372835136.py:4: DtypeWarning: Columns (61,63,65,74,82,83) have mixed types. Specify dtype option on import or set low_memory=False.
  gtdb_metadata = pd.read_csv('/mnt/c/Users/zhenh/tax_data/bac120_metadata.tsv',sep='\t')


In [20]:
gtdb_metadata

,accession,ambiguous_bases,checkm_completeness,checkm_contamination,checkm_marker_count,checkm_marker_lineage,checkm_marker_set_count,checkm_strain_heterogeneity,coding_bases,coding_density,...,ssu_silva_blast_align_len,ssu_silva_blast_bitscore,ssu_silva_blast_evalue,ssu_silva_blast_perc_identity,ssu_silva_blast_subject_id,ssu_silva_taxonomy,total_gap_length,trna_aa_count,trna_count,trna_selenocysteine_count
0,RS_GCF_001206855.1,14,99.60,0.91,761,g__Streptococcus (UID584),238,50.00,1827337,86.246727,...,1543,2844,0,99.935,CP028436.1781495.1783046,Bacteria;Firmicutes;Bacilli;Lactobacillales;St...,21,19,35,0
1,RS_GCF_001108865.1,13,99.39,0.34,761,g__Streptococcus (UID584),238,33.33,1852015,86.277715,...,1543,2844,0,99.935,AKVY01000001.1976120.1977665,Bacteria;Firmicutes;Bacilli;Lactobacillales;St...,0,19,38,0
2,RS_GCF_000730125.1,30,100.00,0.92,476,f__Bifidobacteriaceae (UID1462),217,0.00,2212698,86.271453,...,1522,2811,0,100,CP008885.1576966.1578495,Bacteria;Actinobacteriota;Actinobacteria;Bifid...,619,19,55,0
3,RS_GCF_000950275.1,0,99.45,0.53,921,f__Vibrionaceae (UID4865),366,0.00,4128151,84.994806,...,655,1210,0,100,KY272023.1.1437,Bacteria;Proteobacteria;Gammaproteobacteria;En...,0,17,77,1
4,RS_GCF_000468955.1,0,100.00,0.00,472,o__Lactobacillales (UID543),265,0.00,2107260,86.823994,...,1545,2848,0,99.935,CP003157.2105633.2107180,Bacteria;Firmicutes;Bacilli;Lactobacillales;St...,0,19,60,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394927,GB_GCA_934776845.1,0,91.57,0.00,149,k__Bacteria (UID2329),89,0.00,779596,89.630474,...,1490,1655,0,87.047,FJ951868.1.1481,Bacteria;Firmicutes;Clostridia;Christensenella...,0,19,41,0
394928,GB_GCA_023647245.1,4609,99.97,0.13,1162,f__Enterobacteriaceae (UID5121),336,0.00,4707023,87.866887,...,1315,2396,0,99.316,CP033396.457925.459474,Bacteria;Proteobacteria;Gammaproteobacteria;En...,0,19,80,1
394929,GB_GCA_020344015.1,0,93.69,3.30,147,k__Bacteria (UID2495),91,0.00,4223673,87.882712,...,none,none,none,none,none,none,27200,18,46,1
394930,GB_GCA_945889495.1,0,90.94,3.23,247,c__Deltaproteobacteria (UID3216),155,0.00,4966708,87.941933,...,1538,2719,0,98.57,FPLL01004645.16.1553,Bacteria;RCP2-54;metagenome,0,18,46,1


In [3]:
# Change contig name into taxid
import subprocess
from tqdm import tqdm

def accession_number_to_taxid(accession):
    esummary = subprocess.Popen(["esummary", "-db", "nuccore", "-id", accession], stdout=subprocess.PIPE)
    output = subprocess.check_output(["xtract", "-pattern", "DocumentSummary", "-element", "TaxId"], stdin=esummary.stdout)
    esummary.wait()
    if len(output) == 0:
        return None
    return int(output)

taxid = []
for name in tqdm(sylph_results["Contig_name"]):
    accession = name.split(" ")[0]
    taxid.append(accession_number_to_taxid(accession))

sylph_results["tax_id"] = taxid
    

  0%|          | 0/70 [00:00<?, ?it/s]

 29%|██▊       | 20/70 [00:28<01:09,  1.39s/it]WARNING: FAILURE ( Wed May  8 16:30:44 +08 2024 )
nquire -url https://eutils.ncbi.nlm.nih.gov/entrez/eutils/ esummary.fcgi -db nuccore -id JAHAAF010000001.1 -version 2.0 -retmode xml -tool edirect -edirect 14.6 -edirect_os Linux -email zhenhao@GflowZH.
<ERROR>Invalid uid JAHAAF010000001.1 at position= 0</ERROR>
SECOND ATTEMPT
nquire -url https://eutils.ncbi.nlm.nih.gov/entrez/eutils/ esummary.fcgi -db nuccore -id JAHAAF010000001.1 -version 2.0 -retmode xml -tool edirect -edirect 14.6 -edirect_os Linux -email zhenhao@GflowZH.
<ERROR>Invalid uid JAHAAF010000001.1 at position= 0</ERROR>
LAST ATTEMPT
ERROR: FAILURE ( Wed May  8 16:30:49 +08 2024 )
nquire -url https://eutils.ncbi.nlm.nih.gov/entrez/eutils/ esummary.fcgi -db nuccore -id JAHAAF010000001.1 -version 2.0 -retmode xml -tool edirect -edirect 14.6 -edirect_os Linux -email zhenhao@GflowZH.
<?xml version="1.0" encoding="UTF-8" ?>
<!DOCTYPE eSummaryResult PUBLIC "-//NLM//DTD esummary nucc

100.0002

In [29]:
sylph_results["Genome_file"][0]

'gtdb_genomes_reps_r220/database/GCF/000/425/005/GCF_000425005.1_genomic.fna.gz'

In [4]:
from ete3 import NCBITaxa

ncbi = NCBITaxa()


def get_level(ID, level='genus'):
    """
    Find the genus of a given species ID.
    """
    if ID is None:
        return None
        
    # Get the lineage of the species
    try:
        lineage = ncbi.get_lineage(int(ID))
        ranks = ncbi.get_rank(lineage)
    except:
        return None

    #print(ranks)
    for i in ranks:
        if ranks[i] == level:
            return i
        
    return None

In [22]:
species_taxid = []
genus_taxid = []

for i in sylph_results["tax_id"]:
    species_taxid.append(get_level(i, 'species'))
    genus_taxid.append(get_level(i, 'genus'))

sylph_results["species_taxid"] = species_taxid
sylph_results["genus_taxid"] = genus_taxid

In [23]:
sylph_results

,Sample_file,Genome_file,Taxonomic_abundance,Sequence_abundance,Adjusted_ANI,Eff_cov,ANI_5-95_percentile,Eff_lambda,Lambda_5-95_percentile,Median_cov,Mean_cov_geq1,Containment_ind,Naive_ANI,Contig_name,tax_id,species_taxid,genus_taxid
0,sampled_reads.fastq,gtdb_genomes_reps_r220/database/GCF/000/425/00...,5.0287,3.2719,99.68,3.272,NA-NA,HIGH,NA-NA,3,3.272,1989/2194,99.68,NZ_AUKY01000001.1 Solobacterium moorei DSM 229...,1123263.0,102148.0,123375.0
1,sampled_reads.fastq,gtdb_genomes_reps_r220/database/GCF/008/121/49...,2.9827,2.9515,99.04,1.941,98.79-99.18,1.941,1.74-2.60,2,2.693,2206/3444,98.57,NZ_CP043051.1 [Ruminococcus] gnavus ATCC 29149...,411470.0,33038.0,2316020.0
2,sampled_reads.fastq,gtdb_genomes_reps_r220/database/GCF/000/381/52...,2.9664,1.7408,99.86,1.930,99.79-100.00,1.930,1.74-2.06,2,2.382,1724/2089,99.38,NZ_KB906605.1 Peptostreptococcus anaerobius VP...,1035196.0,1261.0,1257.0
3,sampled_reads.fastq,gtdb_genomes_reps_r220/database/GCF/013/009/55...,2.9076,4.5559,99.32,1.892,99.16-99.42,1.892,1.75-2.24,2,2.583,3655/5274,98.82,NZ_CP046176.1 Phocaeicola dorei strain DSM 178...,357276.0,357276.0,909656.0
4,sampled_reads.fastq,gtdb_genomes_reps_r220/database/GCF/001/025/19...,2.7174,1.5740,99.03,1.768,98.91-99.19,1.768,1.52-1.94,2,2.139,1202/1947,98.46,NZ_AP012325.1 Bifidobacterium catenulatum DSM ...,566552.0,1686.0,1678.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,sampled_reads.fastq,gtdb_genomes_reps_r220/database/GCF/000/157/99...,0.3945,0.4321,100.00,0.257,99.64-100.60,0.257,0.21-0.30,1,1.317,856/3688,95.40,NZ_GG657585.1 Holdemania filiformis DSM 12042 ...,545696.0,61171.0,61170.0
66,sampled_reads.fastq,gtdb_genomes_reps_r220/database/GCF/025/150/56...,0.3434,0.1880,100.00,0.223,99.30-100.95,0.223,0.16-0.29,1,1.113,390/1865,95.08,NZ_CP102283.1 Granulicatella adiacens ATCC 491...,638301.0,46124.0,117563.0
67,sampled_reads.fastq,gtdb_genomes_reps_r220/database/GCF/000/154/46...,0.2585,0.1837,99.70,0.168,98.93-100.90,0.168,0.11-0.21,1,1.113,354/2455,93.94,NZ_DS499581.1 Alistipes putredinis DSM 17216 S...,445970.0,28117.0,239759.0
68,sampled_reads.fastq,gtdb_genomes_reps_r220/database/GCF/002/222/59...,0.2569,0.2194,99.09,0.167,98.26-100.21,0.167,0.11-0.21,1,1.097,340/2868,93.35,NZ_CP022413.2 Blautia hansenii DSM 20583 chrom...,537007.0,1322.0,572511.0


In [5]:
# Get the ground truth abundance
from Bio import SeqIO
# Read ground truth file

labels = []
read_length = []
ground_truth_label = "/mnt/c/Users/zhenh/tax_data/sampled_reads.label"
ground_truth_reads = "/mnt/c/Users/zhenh/tax_data/sampled_reads.fastq"

index = 0
for record in SeqIO.parse(ground_truth_reads, "fastq"):
    read_length.append(len(record.seq))

with open(ground_truth_label, 'r') as gt:
    for line in gt:
        labels.append(int(line))

In [8]:
ground_truth_df = pd.DataFrame({"labels": labels, "read_length": read_length})

In [13]:
ground_truth_df

,labels,read_length
0,100886,8663
1,100886,2129
2,100886,3852
3,100886,6186
4,100886,790
...,...,...
71995,901,25452
71996,901,2122
71997,901,11199
71998,901,2373


In [27]:
abundance_df = ground_truth_df.groupby(labels).sum()

In [28]:
abundance_df["read_length"] = abundance_df["read_length"] / abundance_df["read_length"].sum()

In [29]:
species_taxid = []
genus_taxid = []

for i in abundance_df.index:
    species_taxid.append(get_level(i, 'species'))
    genus_taxid.append(get_level(i, 'genus'))

abundance_df["species_taxid"] = species_taxid
abundance_df["genus_taxid"] = genus_taxid

In [30]:
abundance_df

,labels,read_length,species_taxid,genus_taxid
817,817000,0.022463,817,816.0
818,1636000,0.044365,818,816.0
901,901000,0.015952,901,872.0
1261,2522000,0.024758,1261,1257.0
1358,1358000,0.010215,1358,1357.0
...,...,...,...,...
1232452,1232452000,0.020242,1232452,NaN
1232457,1232457000,0.008461,1232457,NaN
1297617,1297617000,0.012899,1297617,1392389.0
1891238,1891238000,0.012329,1891238,NaN


In [ ]:
abundance_df.join()